### Imports

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import xgboost as xgb
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
import os
import joblib
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

### Encoding

In [3]:
# Encoding functions
def one_hot_encode(data, columns):
    """One-hot encodes specified columns in the dataframe."""
    return pd.get_dummies(data, columns=columns, drop_first=True)

def label_encode(data, columns):
    """Label encodes specified columns in the dataframe."""
    for col in columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
    return data

def frequency_encode(data, columns):
    """Frequency encodes specified columns in the dataframe."""
    for col in columns:
        value_counts = Counter(data[col])
        data[col] = data[col].map(lambda x: value_counts[x] / len(data))
    return data

def target_encode(data, target_column, columns):
    """Target encodes specified columns by replacing categories with the mean of the target variable."""
    for col in columns:
        target_mean = data.groupby(col)[target_column].mean()
        data[col] = data[col].map(target_mean)
    return data

### Imputation

In [4]:
# Imputation functions
def impute_mean(data, column):
    data[column].fillna(data[column].mean(), inplace=True)
    return data

def impute_median(data, columns):
    imputer = SimpleImputer(strategy="median")
    data[columns] = imputer.fit_transform(data[columns])
    return data

def impute_mode(data, column):
    data[column].fillna(data[column].mode()[0], inplace=True)
    return data

def impute_knn(data, columns, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    data[columns] = imputer.fit_transform(data[columns])
    return data

def impute_random_forest(data, features, target_column):
    imputer = RandomForestRegressor()
    known_data = data.dropna(subset=[target_column])
    imputer.fit(known_data[features], known_data[target_column])
    missing_idx = data[target_column].isnull()
    data.loc[missing_idx, target_column] = imputer.predict(data.loc[missing_idx, features])
    return data

def impute_regression(data, columns):
    for col in columns:
        features = [c for c in data.columns if c != col]
        model = LinearRegression()
        model.fit(data[features], data[col])
        data.loc[data[col].isnull(), col] = model.predict(data[features].loc[data[col].isnull()])
    return data

### Evaluation

In [5]:
# Regression evaluation metrics
def evaluate_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'Mean Squared Error: {mse}')
    print(f'Mean Absolute Error: {mae}')
    print(f'R^2 Score: {r2}')
    return {'mse': mse, 'mae': mae, 'r2': r2}

# Classification evaluation metrics
def evaluate_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print(f'Confusion Matrix:\n{cm}')
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'confusion_matrix': cm}

### Settings

In [6]:
df = pd.read_csv('mentalhealth.csv')
filename = os.path.splitext(os.path.basename('mentalhealth.csv'))[0]
target_column = 'Depression'
problem_type = 'classification'.lower()

### Get Info

In [7]:
def dataset_summary(data):
    rows, cols = data.shape
    properties = {}
    print(f'{rows} rows, {cols} columns\n')

    for column in data.columns:
        null_count = int(data[column].isna().sum())
        null_percentage = round((null_count / rows) * 100, 2) if null_count > 0 else 0
        unique = data[column].nunique()
        dtype = data[column].dtype

        properties[column] = {
            'type': dtype,
            'null_count': null_count,
            'null_percentage': null_percentage,
            'unique': unique
        }
        print(f'{column}: {dtype}, {null_count} null ({null_percentage}%), {unique} unique')

    return properties

summary = dataset_summary(df)

140700 rows, 20 columns

id: int64, 0 null (0%), 140700 unique
Name: object, 0 null (0%), 422 unique
Gender: object, 0 null (0%), 2 unique
Age: float64, 0 null (0%), 43 unique
City: object, 0 null (0%), 98 unique
Working Professional or Student: object, 0 null (0%), 2 unique
Profession: object, 36630 null (26.03%), 64 unique
Academic Pressure: float64, 112803 null (80.17%), 5 unique
Work Pressure: float64, 27918 null (19.84%), 5 unique
CGPA: float64, 112802 null (80.17%), 331 unique
Study Satisfaction: float64, 112803 null (80.17%), 5 unique
Job Satisfaction: float64, 27910 null (19.84%), 5 unique
Sleep Duration: object, 0 null (0%), 36 unique
Dietary Habits: object, 4 null (0.0%), 23 unique
Degree: object, 2 null (0.0%), 115 unique
Have you ever had suicidal thoughts ?: object, 0 null (0%), 2 unique
Work/Study Hours: float64, 0 null (0%), 13 unique
Financial Stress: float64, 4 null (0.0%), 5 unique
Family History of Mental Illness: object, 0 null (0%), 2 unique
Depression: int64, 0 nu

### Transform

In [8]:
# Transformation function
def transform(data, target_column=None):
    properties = dataset_summary(data)
    new_df = data.copy()

    for column, prop in properties.items():
        # Skip the target column
        if column == target_column:
            le = LabelEncoder()
            new_df[column] = le.fit_transform(new_df[column])
            print(f'Label encoded target column {column}')
            continue

        # Drop columns with high null percentage
        if prop['null_percentage'] > 50:
            new_df.drop(column, axis=1, inplace=True)
            print(f'Dropped {column} due to high null percentage')
            continue

        # Convert numerical types to float
        if prop['type'] == 'int64':
            new_df[column] = new_df[column].astype(float)
            print(f'Converted {column} from int64 to float64')

        # Imputation
        if prop['null_count'] > 0:
            if prop['type'] == 'object':
                new_df = impute_mode(new_df, column)
                print(f'Imputed {column} with mode')
            else:
                new_df = impute_mean(new_df, column)
                print(f'Imputed {column} with mean')

        # Encoding
        if new_df[column].dtype == 'object':
            unique_count = prop['unique']
            if unique_count < 10:
                new_df = one_hot_encode(new_df, [column])
                print(f'One-hot encoded {column}')
            elif 10 <= unique_count <= 50:
                new_df = frequency_encode(new_df, [column])
                print(f'Frequency encoded {column}')
            elif target_column and target_column in new_df.columns:
                new_df = target_encode(new_df, target_column, [column])
                print(f'Target encoded {column}')
            else:
                new_df = frequency_encode(new_df, [column])
                print(f'Frequency encoded {column}')

    return new_df

new_df = transform(df, target_column)
summary = dataset_summary(new_df)

140700 rows, 20 columns

id: int64, 0 null (0%), 140700 unique
Name: object, 0 null (0%), 422 unique
Gender: object, 0 null (0%), 2 unique
Age: float64, 0 null (0%), 43 unique
City: object, 0 null (0%), 98 unique
Working Professional or Student: object, 0 null (0%), 2 unique
Profession: object, 36630 null (26.03%), 64 unique
Academic Pressure: float64, 112803 null (80.17%), 5 unique
Work Pressure: float64, 27918 null (19.84%), 5 unique
CGPA: float64, 112802 null (80.17%), 331 unique
Study Satisfaction: float64, 112803 null (80.17%), 5 unique
Job Satisfaction: float64, 27910 null (19.84%), 5 unique
Sleep Duration: object, 0 null (0%), 36 unique
Dietary Habits: object, 4 null (0.0%), 23 unique
Degree: object, 2 null (0.0%), 115 unique
Have you ever had suicidal thoughts ?: object, 0 null (0%), 2 unique
Work/Study Hours: float64, 0 null (0%), 13 unique
Financial Stress: float64, 4 null (0.0%), 5 unique
Family History of Mental Illness: object, 0 null (0%), 2 unique
Depression: int64, 0 nu

### Modeling

In [9]:
from sklearn.model_selection import GridSearchCV

def regression_model(data, target_column):
    X = data.drop(columns=[target_column])
    y = data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    metrics = evaluate_regression(y_test, y_pred)
    return metrics, best_model

def classification_model(data, target_column):
    X = data.drop(columns=[target_column])
    y = data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.2, 0.3],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    metrics = evaluate_classification(y_test, y_pred)
    return metrics, best_model

# Example usage:
if problem_type == 'regression':
    print('Regression model evaluation: \n')
    metrics, model = regression_model(new_df, target_column)
    print(metrics)
elif problem_type == 'classification':
    print('Classification model evaluation: \n')
    metrics, model = classification_model(new_df, target_column)
    print(metrics)
elif problem_type == 'timeseries':
    pass

Classification model evaluation: 

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   3.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   3.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   3.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   3.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   3.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   3.8s


KeyboardInterrupt: 

In [10]:
# Method 1: Exporting as a pickle file
with open(f'{filename} model.pkl', 'wb') as file:
    pickle.dump(model, file)
print(f"Model saved as {filename} model.pkl")

# Method 2: Exporting as a joblib file
joblib.dump(model, f'{filename} model.joblib')
print(f"Model saved as {filename} model.joblib")

Model saved as mentalhealth model.pkl
Model saved as mentalhealth model.joblib


In [11]:
# Load the test data
test_df = pd.read_csv('test.csv')

# Preprocess the test data
test_df_transformed = transform(test_df)

# Make predictions
predictions = model.predict(test_df_transformed)

# Create a DataFrame with the predictions
predictions_df = pd.DataFrame({'id': test_df['id'], 'Depression': predictions})

# Save the predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)
print("Predictions saved to predictions.csv")

93800 rows, 19 columns

id: int64, 0 null (0%), 93800 unique
Name: object, 0 null (0%), 374 unique
Gender: object, 0 null (0%), 2 unique
Age: float64, 0 null (0%), 44 unique
City: object, 0 null (0%), 68 unique
Working Professional or Student: object, 0 null (0%), 2 unique
Profession: object, 24632 null (26.26%), 64 unique
Academic Pressure: float64, 75033 null (79.99%), 5 unique
Work Pressure: float64, 18778 null (20.02%), 5 unique
CGPA: float64, 75034 null (79.99%), 326 unique
Study Satisfaction: float64, 75033 null (79.99%), 5 unique
Job Satisfaction: float64, 18774 null (20.01%), 5 unique
Sleep Duration: object, 0 null (0%), 31 unique
Dietary Habits: object, 5 null (0.01%), 22 unique
Degree: object, 2 null (0.0%), 87 unique
Have you ever had suicidal thoughts ?: object, 0 null (0%), 2 unique
Work/Study Hours: float64, 0 null (0%), 13 unique
Financial Stress: float64, 0 null (0%), 5 unique
Family History of Mental Illness: object, 0 null (0%), 2 unique
Converted id from int64 to flo